In [1]:
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from lmfit import minimize, Parameters, report_fit
import random
import warnings
import matplotlib.pyplot as plt
import time
import itertools

In [2]:
#Load and prepare data
path_tp = '../../../Data/'
dataTP_05mg = pd.read_csv(path_tp + 'Matthews2017/Digitized/PK/05mg_TP.csv')
dataTP_10mg_ = pd.read_csv(path_tp + 'Matthews2017/Digitized/PK/10mg_TP.csv')
dataTP_10mg_w1 = pd.read_csv(path_tp + 'Grobler2016/Digitized/PK/10mg_TP_W1.csv')
dataTP_10mg_w3 = pd.read_csv(path_tp + 'Grobler2016/Digitized/PK/10mg_TP_W3.csv')
dataTP_30mg_ = pd.read_csv(path_tp + 'Matthews2017/Digitized/PK/30mg_TP.csv')
dataTP_30mg_w1 = pd.read_csv(path_tp + 'Grobler2016/Digitized/PK/30mg_TP_W1.csv')
dataTP_30mg_w3 = pd.read_csv(path_tp + 'Grobler2016/Digitized/PK/30mg_TP_W3.csv')
dataTP_100mg_w1 = pd.read_csv(path_tp + 'Grobler2016/Digitized/PK/100mg_TP_W1.csv')
dataTP_100mg_w3 = pd.read_csv(path_tp + 'Grobler2016/Digitized/PK/100mg_TP_W3.csv')
dataTP_1mg = pd.read_csv(path_tp + 'Matthews2017/Digitized/PK/1mg_TP.csv')
dataTP_2mg = pd.read_csv(path_tp + 'Matthews2017/Digitized/PK/2mg_TP.csv')

dataTP_05mg = dataTP_05mg.dropna(axis='columns')
dataTP_1mg = dataTP_1mg.dropna(axis='columns')
dataTP_2mg = dataTP_2mg.dropna(axis='columns')
dataTP_10mg_ = dataTP_10mg_.dropna(axis='columns')
dataTP_10mg_w1 = dataTP_10mg_w1.dropna(axis='columns')
dataTP_10mg_w3 = dataTP_10mg_w3.dropna(axis='columns')
dataTP_30mg_ = dataTP_30mg_.dropna(axis='columns')
dataTP_30mg_w1 = dataTP_30mg_w1.dropna(axis='columns')
dataTP_30mg_w3 = dataTP_30mg_w3.dropna(axis='columns')
dataTP_100mg_w1 = dataTP_100mg_w1.dropna(axis='columns')
dataTP_100mg_w3 = dataTP_100mg_w3.dropna(axis='columns')

dataTP_05mg.columns = ['time','conc']
dataTP_1mg.columns = ['time','conc']
dataTP_2mg.columns = ['time','conc']
dataTP_10mg_.columns = ['time','conc']
dataTP_10mg_w1.columns = ['time','conc']
dataTP_10mg_w3.columns = ['time','conc']
dataTP_30mg_.columns = ['time','conc']
dataTP_30mg_w1.columns = ['time','conc']
dataTP_30mg_w3.columns = ['time','conc']
dataTP_100mg_w1.columns = ['time','conc']
dataTP_100mg_w3.columns = ['time','conc']

dataTP_10mg = pd.concat([dataTP_10mg_, dataTP_10mg_w1, dataTP_10mg_w3]).sort_values(by=['time']).reset_index(drop=True)
dataTP_30mg = pd.concat([dataTP_30mg_, dataTP_30mg_w1, dataTP_30mg_w3]).sort_values(by=['time']).reset_index(drop=True)
dataTP_100mg = pd.concat([dataTP_100mg_w1, dataTP_100mg_w3]).sort_values(by=['time']).reset_index(drop=True)

dataTP_10mg.time.iloc[0] = abs(dataTP_10mg.time.iloc[0])

datalist_TP = [dataTP_10mg, dataTP_30mg, dataTP_100mg, dataTP_2mg, dataTP_1mg, dataTP_05mg]
data_TP = pd.concat([dataTP_10mg, dataTP_30mg, dataTP_100mg, dataTP_2mg, dataTP_1mg, dataTP_05mg]) #assembled data

# convert to unit nM
data_TP.conc = 6 - np.log10(180) + data_TP.conc.tolist()
for n in range(len(datalist_TP)): #to nM    
    datalist_TP[n].conc = 6 - np.log10(180) + datalist_TP[n].conc.tolist()

data_TP.time.iloc[0] = 0.2

In [9]:
#Compertment model MMK

def model_TP_MMK(t, z, params):
    #parameters to estimate
    Vmax = params['Vmax'].value
    km = params['km'].value
    k30 = params['k30'].value
    Z0 = z[0]; Z1 = z[1]; Z2 = z[2]; Z3 = z[3]
    dZ0 = -ka*Z0
    dZ1 = (ka/Vc)*Z0 - k10*Z1 - k12*Z1 + k21*Z2
    dZ2 = k12*Z1 - k21*Z2
    dZ3 = (Vmax*Z1)/(km+Z1) - k30*Z3
    d = [dZ0,dZ1,dZ2,dZ3]
    return d

def solve_ode_TP(z, data_mat, params):
    '''
    Solve the ODE-system for
    z = initial state of the system
    data_mat = assembled data set
    params = parameters to estimate
    
    return list of drug concentrations from intracellular compartment (Z3)
    '''
    ind = data_mat.index.tolist() #get indices
    offset = 0 #each data set is continuously numbered
    logZ3 = []
    for j in range(len(z)):
        i = 0 
        while (ind[i+offset] == i): #while index euqlas offset
            i += 1
            if i+offset > len(ind)-1: #break if end of list is reached
                break
        offset += i #add current index to offset
        t_obs = data_mat.time[offset-i:offset].tolist()
        res = solve_ivp(model_TP_MMK, (tstart,t_obs[-1:][0]), z[j], t_eval=t_obs,args=(params,))
        #print(res)
        logZ3.append(np.log10(res.y[3])) #intracellular compartment
    return list(itertools.chain.from_iterable(logZ3))

def solve_ode_simple(z, t_obs, params): #solve ode for a single data set
    res = solve_ivp(model_TP_MMK, (tstart,t_obs[-1:]), z, t_eval=t_obs,args=(params,))
    return res

def residual_TP(params, z, data_mat):
    logZ3 = solve_ode_TP(z, data_mat, params)
    return np.power(np.subtract(logZ3,data_mat.conc.tolist()),2)

def new_params(): #sample parameters
    global parameters
    vmax = random.uniform(0,500000)
    vkm = random.uniform(0,500000)
    v30 = random.uniform(0,1)
    parameters = Parameters()
    parameters.add('Vmax', value = vmax, min = 0, max = 500000)
    parameters.add('km', value = vkm, min = 0, max = 500000)
    parameters.add('k30', value = v30, min = 0, max = 1)

In [10]:
#Global parameters
dose = 3410 #nM

tstart = 0 #initial and final time of the system

#initial states of the system    
z0 = [[10*dose,0,0,0],[30*dose,0,0,0], [100*dose,0,0,0],[2*dose,0,0,0],[dose,0,0,0],[0.5*dose,0,0,0]] 

#Estimated PK plasma parameters
ka = 45.4382
k10 =  0.2355
k12 =  0.1750
k21 =  0.0259
Vc =  162.69

In [11]:
threshold = 0 #user-defined threshold
count = 0; maxcount = 1 #initialand maximal number of iterations
VmaxList = []; kmList = []; k30List = []; aicList = []; rssList = []

while count < maxcount:
    new_params() #sample parameters
    success = True
    try:
        result = minimize(residual_TP, parameters, args=(z0, data_TP), method = 'leastsq') #Levenberg-Marquardt algorithm
        #report_fit(result)
        if result.aic < threshold: 
            rss = np.sum(residual_TP(result.params,z0,data_TP))
            aicList.append(result.aic)
            rssList.append(rss)
            kmList.append(result.params['km'].value)
            k30List.append(result.params['k30'].value)
            VmaxList.append(result.params['Vmax'].value)
        else: 
            success = False
            print('AIC too large')
            new_params()
    except:
        print('an unexpected error occurred!')
        success = False
        new_params()
       
    if success:
        count += 1
        print(); print('run ',count,' finished') 


run  1  finished


In [12]:
# evaluation
dfResult = pd.DataFrame(list(zip(aicList,rssList,VmaxList, kmList,k30List)),
                        columns =['AIC','RSS','Vmax', 'km', 'k30']) 
print(); print(dfResult)
#dfResult.to_excel('TP_Parameters_MMK.xlsx') #store result


          AIC       RSS          Vmax            km       k30
0 -444.661841  4.364662  499999.83544  11026.163333  0.009827
